<a href="https://colab.research.google.com/github/taniokah/DL-Basic-Seminar/blob/master/AI%E3%83%96%E3%83%BC%E3%83%A0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AIブーム分析

https://ohke.hateblo.jp/entry/2019/06/01/120000
https://qiita.com/a2kiti/items/a93d762bd89a84316c12

1. まず、Sudachiのword2vecモデルをダウンロードする
2. 対象となる新聞記事を読み込む
3. 新聞記事からAIに近い単語を上位10個ずつ抽出する
4. 単語ベクトルとその頻度のリストを作る
5. 時系列トピックモデルを用いて分析する


In [1]:
!wget https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz

--2020-01-28 10:50:54--  https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz
Resolving object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)... 157.7.224.17
Connecting to object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)|157.7.224.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5212195768 (4.9G) [application/x-www-form-urlencoded]
Saving to: ‘nwjc.sudachi_full_abc_w2v.20190314.tar.gz’

nwjc.sudachi_full_a 100%[===================>]   4.85G  19.1MB/s    in 4m 27s  

2020-01-28 10:55:22 (18.6 MB/s) - ‘nwjc.sudachi_full_abc_w2v.20190314.tar.gz’ saved [5212195768/5212195768]



In [2]:
!tar -zxvf ./nwjc.sudachi_full_abc_w2v.20190314.tar.gz

nwjc_sudachi_full_abc_w2v/
nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt
nwjc_sudachi_full_abc_w2v/LICENSE


In [3]:
!ls -l nwjc_sudachi_full_abc_w2v/

total 12229548
-rw-rw-r-- 1 1008 1008       11420 Mar 20  2019 LICENSE
-rw-rw-r-- 1 1008 1008 12523039651 Mar 20  2019 nwjc.sudachi_full_abc_w2v.txt


In [4]:
!head -n2 nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt

3644628 300
の -0.08509623 -0.02170456 -0.00595318 0.17987297 -0.02470185 -0.06039717 0.16489118 0.18576832 -0.13870764 -0.17454691 0.08653757 -0.11530939 -0.0357993 0.0308698 -0.11082912 0.04128339 0.00738431 0.00342017 -0.00673456 0.03348965 -0.18876095 0.05184764 0.13508447 0.24196455 -0.07351201 -0.07903086 -0.01973874 -0.24246444 0.03141621 -0.11729659 -0.04126292 -0.20383213 -0.03109288 0.06765312 -0.15020953 -0.11498244 0.03988069 0.20287995 -0.14308368 -0.05135961 -0.15592268 0.05364651 -0.11668575 0.0505892 -0.04575419 0.03296537 -0.03033004 -0.0481239 0.13578212 -0.03111291 -0.15257519 -0.10075521 -0.07440718 -0.12524064 -0.14464405 -0.00155862 0.11825287 0.01658496 0.27135071 -0.07724451 0.01130517 0.03584538 0.01659033 0.19819288 0.01558947 0.07239497 -0.05149494 0.08688559 -0.03712051 -0.07294089 -0.00820937 -0.00907754 -0.03328617 -0.09273465 0.07537545 -0.05137397 0.07578564 -0.03124459 0.05484971 -0.07136369 -0.02457538 0.15481928 0.11072636 -0.00225826 -0.04312619 -0.09

In [0]:
#!pip install gensim

In [6]:
# モデルのロード
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

nwjc_model = KeyedVectors.load_word2vec_format('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt', binary=False)

#nwjc_model = KeyedVectors.load_word2vec_format(
#    datapath('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt'),
#    binary=False
#)

# 語数, 次元数
print(len(nwjc_model.vocab), nwjc_model.vector_size)  # 3644628 300

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


3644628 300


In [7]:
print(nwjc_model.most_similar('平成', topn=5))
# [('年度', 0.8092531561851501),
#  ('平成元年', 0.7409119009971619),
#  ('同年', 0.6042426824569702),
#  ('基本計画', 0.5917632579803467),
#  ('別紙', 0.590812087059021)]

print(nwjc_model.most_similar(positive=['兄弟', '女'], negative=['男'], topn=5))
# [('姉妹', 0.7687463164329529),
#  ('兄妹', 0.7484474182128906),
#  ('姉弟', 0.715774416923523),
#  ('双子', 0.642593264579773),
#  ('妹', 0.6402267217636108)]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('年度', 0.8092531561851501), ('平成元年', 0.7409119009971619), ('同年', 0.6042426824569702), ('基本計画', 0.5917632579803467), ('別紙', 0.590812087059021)]
[('姉妹', 0.7687463164329529), ('兄妹', 0.7484474182128906), ('姉弟', 0.715774416923523), ('双子', 0.642593264579773), ('妹', 0.6402267217636108)]
